# [4.7 Solving CSPs using Stochastic Local Search](http://artint.info/2e/html/ArtInt2e.Ch4.S7.html)

- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=63) (page 63)

## About

For large CSPs, systematically searching through the space of assignments of values to variables will prove unviable. This section considers a method intended to work in these very large spaces, __stochastic local search__. Stochastic local search does not systematically search the whole search space but they are designed to find solutions quickly on average. It does not guarantee that a solution will be found even if one exists, and so it is not able to prove that no solution exists. It is often the method of choice for applications where solutions are known to exist or are very likely to exist.



## Instructions

Each section header contains a link to the corresponding chapter in the accompanying textbook, and an "Implementation Details" link provided throughout tells you how the implementation works. Before using this notebook, make sure you have followed the [installation instructions](https://aispace2.github.io/AISpace2/install.html) beforehand.

You can run each cell by selecting it and pressing *Ctrl+Enter* in Windows or *Shift+Return* in Mac. Alternatively, you can click the *Play* button in the toolbar, to the left of the stop button. 

For more information, including how the code in this notebook differs from that in [AIPython](aipython.org), check out the [Reference](https://aispace2.github.io/AISpace2/reference.html).

In [ ]:
# Run this to import pre-defined problems
from aipython.cspProblem import csp_simple1, csp_simple2, csp_extended, csp_crossword1, csp_crossword2, csp_crossword2d

In [ ]:
from aipython.cspSLS import SLSearcher
sls = SLSearcher(csp=csp_simple1)
sls.csp_id = 'sls' # For button layout, 'sls'=stochastic local seach, 'con'=arc consistency

# Visualization options
sls.sleep_time = 0.2 # The time, in seconds, between each step in auto solving
sls.line_width = 4.0 # The thickness of edges
sls.text_size = 15 # The fontsize of the text
sls.detail_level = 1 # 0=no text, 1=truncated text, 2=full text

# Display the widget
display(sls)
sls.search(max_steps=500, prob_best=0, prob_anycon=1.0)
"""
prob_best is the probability of selecting a best variable (one involving the most conflicts)
prob_anycon is the probability that the any-conflict strategy is used,
    which selects a variable at random that is in a conflict, 
    assuming that it is not picking a best variable
"""

# Plotting Runtime Distributions
- [Implementation Details](http://artint.info/AIPython/aipython.pdf#page=69) (page 69)

Due to the stochastic nature of local search, multiple runs over the same problem will yield different results. Instead, we look at the runtime distribution - the number of runs that find a solution within a given number of steps.

In [ ]:
from aipython.cspSLSPlot import Runtime_distribution
p = Runtime_distribution(csp=csp_extended, xscale='log') # xscale is either 'linear' or 'log'
p.plot_run(num_runs=100, max_steps=1000, prob_best=0, prob_anycon=1.0)
p.plot_run(num_runs=100, max_steps=1000, prob_best=0.7, prob_anycon=1.0)